# Regrid Kristen's CESM chlorophyll output for use by TE (regrid to JRA mesh)
# This was run on NCAR's derecho where all the model output is stored.

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xesmf as xesmf
import pandas as pd
from netCDF4 import Dataset, MFDataset
import os
import glob

In [2]:
#---
# pick a random JRA file to get lat/lon info
#---

lr_grid = xr.open_dataset('/glade/p/cesmdata/cseg/inputdata/ocn/jra55/v1.5_noleap/JRA.v1.5.v_10.TL319.2021.220505.nc',\
                          decode_times=False)['v_10'].isel(time=0)
lr_grid = lr_grid.drop_vars('time')
lon_jra = lr_grid['longitude'].values
lat_jra = lr_grid['latitude'].values
#lr_grid

In [3]:
#---
# load Kristen's data
#---

path1 = '/glade/campaign/cgd/oce/projects/FOSI_BGC/HR/g.e22.TL319_t13.G1850ECOIAF_JRA_HR.4p2z.001/ocn/proc/tseries/day_1/'
file_chl = 'g.e22.TL319_t13.G1850ECOIAF_JRA_HR.4p2z.001.pop.h.nday1.ECOSYS_IFRAC_2.19890102-19900101.nc'

var = 'ECOSYS_IFRAC_2'
ff = xr.open_dataset(path1+file_chl)[var]#.isel(z_t=0)
ff = ff.rename({'TLONG': 'longitude','TLAT': 'latitude'})
ff = ff.drop_vars('ULONG')
ff = ff.drop_vars('ULAT')
ff = ff.drop_vars('time')
#ff


In [4]:
#regridder_jra = xesmf.Regridder(ff, lr_grid, 'bilinear') 
#ds_rg = regridder_jra(ff)
#ds_rg.to_netcdf('/glade/work/cnissen/CESM_chl_regridded/test.nc')
#ds_rg
#data_to_save = np.copy(ds_rg.values)
#print(data_to_save.shape)

In [5]:
#-----
# loop over all years and save as netcdf file 
#-----

savepath = '/glade/work/cnissen/CESM_HIGH_RES_seaice_regridded/'
save_netcdf = True

path1 = '/glade/campaign/cgd/oce/projects/FOSI_BGC/HR/g.e22.TL319_t13.G1850ECOIAF_JRA_HR.4p2z.001/ocn/proc/tseries/day_1/'
var1  = 'ECOSYS_IFRAC_2'

var   = 'ECOSYS_IFRAC_2'
var_description = 'sea ice concentration'
fv = -999

year_list = np.arange(1978,2018+1,1)

for yy in range(18,len(year_list)):
    print('Load year',year_list[yy])
    
    # in the folder, find the file that has "var1.year_list[yy]" in its name
    file_chl = glob.glob(path1+'*'+var1+'.'+str(year_list[yy])+'*')[0]
    ff = xr.open_dataset(file_chl)[var1]#.isel(z_t=0)
    ff = ff.rename({'TLONG': 'longitude','TLAT': 'latitude'})
    ff = ff.drop_vars('ULONG')
    ff = ff.drop_vars('ULAT')
    ff = ff.drop_vars('time')
    # create mapping file
    regridder_jra = xesmf.Regridder(ff, lr_grid, 'bilinear') 
    # regrid field
    ds_rg = regridder_jra(ff)
    # create numpy array
    data_to_save = np.copy(ds_rg.values)
    # close file
    ff.close()
    
    if save_netcdf:
        netcdf_name = var+'_hr_JRA_grid_'+str(year_list[yy])+'-01-01.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.contact = 'Cara Nissen, cara.nissen@colorado.edu'
            w_nc_fid.source_file = path1+file_chl
            w_nc_fid.script      = '/glade/u/home/cnissen/scripts/regrid_CESM_HIGH_RES_sea_ice_output_JRA_mesh.ipynb'
            # create dimension & variable
            w_nc_fid.createDimension('time', data_to_save.shape[0]) 
            w_nc_fid.createDimension('latitude', data_to_save.shape[1]) 
            w_nc_fid.createDimension('longitude', data_to_save.shape[2]) 
            w_nc_var1 = w_nc_fid.createVariable(var, 'f4',('time','latitude','longitude'),fill_value=fv)
            w_nc_var1.description = var_description
            w_nc_var1.units = 'deg C'
            w_nc_var1 = w_nc_fid.createVariable('latitude', 'f4',('latitude'),fill_value=fv)
            w_nc_var1.description = 'Latitude'
            w_nc_var1.units = 'deg N'
            w_nc_var1 = w_nc_fid.createVariable('longitude', 'f4',('longitude'),fill_value=fv)
            w_nc_var1.description = 'Longitude (0:360)'
            w_nc_var1.units = 'deg E'
            w_nc_fid.close()
                     
        data_to_save[np.isnan(data_to_save)] = fv
        data_to_save[data_to_save==0] = fv

        # get rid of white line!
        data_to_save[:,:,568] = data_to_save[:,:,567]
        data_to_save[:,:,569] = data_to_save[:,:,570]

        w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
        w_nc_fid.variables[var][:,:] = data_to_save
        w_nc_fid.variables['latitude'][:] = lat_jra
        w_nc_fid.variables['longitude'][:] = lon_jra
        w_nc_fid.close()  

        print('Succesfully saved year '+str(year_list[yy]))
                
print('done')


Load year 1996
Create file /glade/work/cnissen/CESM_HIGH_RES_seaice_regridded/ECOSYS_IFRAC_2_hr_JRA_grid_1996-01-01.nc
Succesfully saved year 1996
Load year 1997
Create file /glade/work/cnissen/CESM_HIGH_RES_seaice_regridded/ECOSYS_IFRAC_2_hr_JRA_grid_1997-01-01.nc
Succesfully saved year 1997
Load year 1998
Create file /glade/work/cnissen/CESM_HIGH_RES_seaice_regridded/ECOSYS_IFRAC_2_hr_JRA_grid_1998-01-01.nc
Succesfully saved year 1998
Load year 1999
Create file /glade/work/cnissen/CESM_HIGH_RES_seaice_regridded/ECOSYS_IFRAC_2_hr_JRA_grid_1999-01-01.nc
Succesfully saved year 1999
Load year 2000
Create file /glade/work/cnissen/CESM_HIGH_RES_seaice_regridded/ECOSYS_IFRAC_2_hr_JRA_grid_2000-01-01.nc
Succesfully saved year 2000
Load year 2001
Create file /glade/work/cnissen/CESM_HIGH_RES_seaice_regridded/ECOSYS_IFRAC_2_hr_JRA_grid_2001-01-01.nc
Succesfully saved year 2001
Load year 2002
Create file /glade/work/cnissen/CESM_HIGH_RES_seaice_regridded/ECOSYS_IFRAC_2_hr_JRA_grid_2002-01-01.nc

In [6]:
#ds_rg.isel(time=0).plot()

In [7]:
#----
# TESTING: get rid of vertical line of missing values!
#----

testing = False
if testing: 
    path1 = '/glade/work/cnissen/CESM_chl_regridded//'
    file_chl = 'Anomalies_totChl_JRA_grid_1958-01-01.nc'
    
    var  = 'totChl'
    ff   = xr.open_dataset(path1+file_chl)#[var]#.isel(z_t=0)
    data = ff[var].values 
    ff.close()
    
    print(data.shape)
    
   # data[:,:,568] = data[:,:,567]
   # data[:,:,569] = data[:,:,570]
    
    fig = plt.figure(figsize=(10,5))
    plt.pcolor(data[1,:,:],cmap=plt.cm.viridis,vmin=0,vmax=0.4)
    plt.colorbar()
    plt.show()
